## Mapping the Data
In `02_st-cleaing-5yr.ipynb` we found that there are some hotspots for street cleaning violations. Here let's map them out.  
Alternatively, I also marked hotposts on [Google MyMap](https://www.google.com/maps/d/edit?mid=1hqpDiDN_2dWwhyBaKwjsPb2qxoinSww&usp=sharing) for better view with analysis neighborhood.

### Import Data

In [1]:
import pandas as pd
import requests

In [2]:
# Geocoding addresses from Pandas
# Import citation_st_cleaning_5yr.csv
st_cleaning_map = pd.read_csv('citation_st_cleaning_5yr.csv',
                              dtype={'Citation Number': object} ,
                              parse_dates=['Citation Issued DateTime'])

In [3]:
st_cleaning_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2373574 entries, 0 to 2373573
Data columns (total 16 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   Citation Number               object        
 1   Citation Issued DateTime      datetime64[ns]
 2   Violation                     object        
 3   Violation Description         object        
 4   Citation Location             object        
 5   Vehicle Plate State           object        
 6   Vehicle Plate                 object        
 7   Fine Amount                   float64       
 8   Date Added                    object        
 9   geom                          object        
 10  Neighborhoods                 float64       
 11  SF Find Neighborhoods         float64       
 12  Current Police Districts      float64       
 13  Current Supervisor Districts  float64       
 14  Analysis Neighborhoods        float64       
 15  Issued Year                   in

In [4]:
# Add new column with complete address for 'Citation Location' for geocoding
st_cleaning_map['Location_new'] = st_cleaning_map['Citation Location'].astype(str) + ', SAN FRANCISCO, USA'

In [5]:
st_cleaning_map.head(10)

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
0,949367440,2022-03-23 02:33:00,TRC7.2.22,STR CLEAN,720 TURK ST,CA,8U09234,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"720 TURK ST, SAN FRANCISCO, USA"
1,949375335,2022-03-24 09:30:00,TRC7.2.22,STR CLEAN,19 RIVAS AVE,CA,8GOA162,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"19 RIVAS AVE, SAN FRANCISCO, USA"
2,949395613,2022-03-21 13:05:00,TRC7.2.22,STR CLEAN,1405 BAKER ST,CA,5JZT981,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"1405 BAKER ST, SAN FRANCISCO, USA"
3,949411481,2022-03-29 00:15:00,TRC7.2.22,STR CLEAN,420 NATOMA ST,CA,8DEJ983,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"420 NATOMA ST, SAN FRANCISCO, USA"
4,949412855,2022-03-22 12:25:00,TRC7.2.22,STR CLEAN,717 WALLER ST,CA,6GOF707,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"717 WALLER ST, SAN FRANCISCO, USA"
5,894375451,2018-02-20 10:15:00,TRC7.2.22,STR CLEAN,3898 WASHINGTON ST,CA,ELBA227,73.0,08/10/2021 12:00:00 AM,POINT (-122.45708101499997 37.788961003000054),10.0,10.0,8.0,6.0,31.0,2018,"3898 WASHINGTON ST, SAN FRANCISCO, USA"
6,895929086,2018-03-29 09:35:00,TRC7.2.22,STR CLEAN,812 SILLIMAN ST,CA,7TGY978,73.0,08/10/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"812 SILLIMAN ST, SAN FRANCISCO, USA"
7,938910103,2021-05-03 12:13:00,TRC7.2.22,STR CLEAN,1820 25TH ST,CA,6HKN775,83.0,08/08/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2021,"1820 25TH ST, SAN FRANCISCO, USA"
8,900852466,2018-08-13 08:24:00,TRC7.2.22,STR CLEAN,173 ARKANSAS ST,CA,56907K2,76.0,02/13/2022 12:00:00 AM,POINT (-122.39854476899995 37.76402533000004),54.0,54.0,1.0,9.0,26.0,2018,"173 ARKANSAS ST, SAN FRANCISCO, USA"
9,900915735,2018-08-17 09:40:00,TRC7.2.22,STR CLEAN,224 GARFIELD ST,CA,7T62119,76.0,02/13/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"224 GARFIELD ST, SAN FRANCISCO, USA"


### Replies by SFMTA
Data cleaned in 02_st-cleaning-5yr.ipynb.

1. Citations with $0.0 amount means that the citations were voided after they were written. This often happens if the vehicles moves before the officer finishes writing the citation.
2. There are citations given between March 17, 2020 and June 14, 2020. Most street cleaning locations for these citations are in the morning and the anouncement for not issueing citation is in the afternoon of March 17, 2020. The SFMTA refunded citations issued that day and voided that were not paid.

### Clean Location Data
While there are `1432876` rows lack with 'geom' information, there are some citations that have no 'Citation Location' but provided with 'geom'. Use `geopy` to geocode those rows.

In [6]:
# See rows that is null in the 'geom' column
st_cleaning_map[
    st_cleaning_map['geom'].isna()
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
1742669,892597812,2018-01-02 00:11:00,TRC7.2.22,STR CLEAN,755 FOLSOM ST,CA,7EJP872,73.0,02/03/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"755 FOLSOM ST, SAN FRANCISCO, USA"
1822320,892746256,2018-01-02 02:51:00,TRC7.2.22,STR CLEAN,903 SANSOME ST,CA,7HUP071,73.0,04/30/2020 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"903 SANSOME ST, SAN FRANCISCO, USA"
1819891,892361131,2018-01-02 06:05:00,TRC7.2.22,STR CLEAN,1031 O'FARRELL ST,NaN,NaN,73.0,12/08/2020 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"1031 O'FARRELL ST, SAN FRANCISCO, USA"
1807039,892607505,2018-01-02 06:06:00,TRC7.2.22,STR CLEAN,3106 MISSION ST,CA,7GSJ377,73.0,08/05/2018 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"3106 MISSION ST, SAN FRANCISCO, USA"
1790294,892359042,2018-01-02 06:08:00,TRC7.2.22,STR CLEAN,383 PARNASSUS AVE,CA,7LWG201,73.0,03/07/2018 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2018,"383 PARNASSUS AVE, SAN FRANCISCO, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035060,957285862,2022-11-16 12:44:00,TRC7.2.22,STR CLEAN,1601 BRODERICK STREET,CA,6XCJ079,87.0,11/22/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"1601 BRODERICK STREET, SAN FRANCISCO, USA"
1038790,957212222,2022-11-16 12:45:00,TRC7.2.22,STR CLEAN,1501 BRODERICK ST,CA,7JUR685,87.0,11/22/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"1501 BRODERICK ST, SAN FRANCISCO, USA"
1027288,957148264,2022-11-16 13:12:00,TRC7.2.22,STR CLEAN,1160 REVERE AVENUE,CA,5P65273,87.0,11/22/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"1160 REVERE AVENUE, SAN FRANCISCO, USA"
1035850,957240712,2022-11-16 13:18:00,TRC7.2.22,STR CLEAN,2444 15TH AVE,CO,FLT388S,87.0,11/22/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"2444 15TH AVE, SAN FRANCISCO, USA"


In [7]:
# There are 11 locations with NaN but have geometry location data.
st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
1730561,1005474271,2018-08-30 07:21:00,TRC7.2.22,STR CLEAN,NaN,CA,6TZX367,76.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2018,"nan, SAN FRANCISCO, USA"
1374851,1005195365,2019-02-13 12:06:00,TRC7.2.22,STR CLEAN,NaN,CA,7AJE508,76.0,04/11/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
1568559,1005518850,2019-04-15 21:47:00,TRC7.2.22,STR CLEAN,NaN,NaN,NaN,76.0,12/04/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
2160991,1005015281,2019-08-02 09:13:00,TRC7.2.22,STR CLEAN,NaN,TX,LVK7849,79.0,07/24/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
2158680,1005773801,2019-08-02 12:15:00,TRC7.2.22,STR CLEAN,NaN,CA,6DQT925,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
1332803,1005771734,2019-08-08 13:14:00,TRC7.2.22,STR CLEAN,NaN,CA,8HED311,79.0,02/21/2022 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
1822634,1005819054,2019-09-12 07:28:00,TRC7.2.22,STR CLEAN,NaN,CA,8DKL101,79.0,10/15/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
1492371,1005224194,2019-10-03 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,NaN,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
1659202,1005938765,2019-11-12 12:05:00,TRC7.2.22,STR CLEAN,NaN,CA,6WJP697,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
632292,926241094,2019-12-17 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,6MLP864,79.0,11/23/2021 12:00:00 AM,POINT (-122.43454601699995 37.76714601000003),28.0,28.0,7.0,5.0,5.0,2019,"nan, SAN FRANCISCO, USA"


In [8]:
# Save data that needs to geocode to another dataframe. Save as .csv in case as well.
st_cleaning_geocode = st_cleaning_map[
    (st_cleaning_map['geom'].notnull()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime']).reset_index()

In [9]:
# Export dataframe as .csv
st_cleaning_geocode.to_csv('st_cleaning_geocode.csv', index=False)

### Missing Location Infos
Yet to be solved. Might considered delete them in 02_st-cleaning-5yr.ipynb

In [10]:
# There are 3 rows of data that have no location data at all
st_cleaning_map[
    (st_cleaning_map['geom'].isna()) &
    (st_cleaning_map['Citation Location'].isna())
].sort_values(['Citation Issued DateTime'])

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
474578,1005977910,2020-09-04 07:20:00,TRC7.2.22,STR CLEAN,NaN,CA,6LZC897,79.0,10/17/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2020,"nan, SAN FRANCISCO, USA"
545686,1006049240,2020-09-09 10:10:00,TRC7.2.22,STR CLEAN,NaN,CA,5DTY745,79.0,11/13/2020 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2020,"nan, SAN FRANCISCO, USA"
191501,1006046602,2021-02-17 10:14:00,TRC7.2.22,STR CLEAN,NaN,CA,KNGJSRI,83.0,04/21/2021 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2021,"nan, SAN FRANCISCO, USA"


### Reverse Geocoing

In [11]:
!pip install censusgeocode

In [12]:
import censusgeocode as cg
import json
import geopandas as gpd

In [13]:
# Import st_cleaning_geocode.csv for geocoding
geocode_11 = pd.read_csv('st_cleaning_geocode.csv',
                              dtype={'Citation Number': object} ,
                              parse_dates=['Citation Issued DateTime'])

In [14]:
geocode_11

,index,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
0,1730561,1005474271,2018-08-30 07:21:00,TRC7.2.22,STR CLEAN,NaN,CA,6TZX367,76.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2018,"nan, SAN FRANCISCO, USA"
1,1374851,1005195365,2019-02-13 12:06:00,TRC7.2.22,STR CLEAN,NaN,CA,7AJE508,76.0,04/11/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
2,1568559,1005518850,2019-04-15 21:47:00,TRC7.2.22,STR CLEAN,NaN,NaN,NaN,76.0,12/04/2019 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
3,2160991,1005015281,2019-08-02 09:13:00,TRC7.2.22,STR CLEAN,NaN,TX,LVK7849,79.0,07/24/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
4,2158680,1005773801,2019-08-02 12:15:00,TRC7.2.22,STR CLEAN,NaN,CA,6DQT925,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
5,1332803,1005771734,2019-08-08 13:14:00,TRC7.2.22,STR CLEAN,NaN,CA,8HED311,79.0,02/21/2022 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
6,1822634,1005819054,2019-09-12 07:28:00,TRC7.2.22,STR CLEAN,NaN,CA,8DKL101,79.0,10/15/2021 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
7,1492371,1005224194,2019-10-03 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,NaN,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
8,1659202,1005938765,2019-11-12 12:05:00,TRC7.2.22,STR CLEAN,NaN,CA,6WJP697,79.0,04/30/2020 12:00:00 AM,POINT (-122.41963999999996 37.777120000000025),21.0,21.0,4.0,10.0,36.0,2019,"nan, SAN FRANCISCO, USA"
9,632292,926241094,2019-12-17 00:00:00,TRC7.2.22,STR CLEAN,NaN,CA,6MLP864,79.0,11/23/2021 12:00:00 AM,POINT (-122.43454601699995 37.76714601000003),28.0,28.0,7.0,5.0,5.0,2019,"nan, SAN FRANCISCO, USA"


In [15]:
geocode_11['geom'].value_counts()

POINT (-122.41963999999996 37.777120000000025)    10
POINT (-122.43454601699995 37.76714601000003)      1
Name: geom, dtype: int64

In [16]:
# Since there are only two long and lat to code, do it manually
## POINT (-122.43454601699995 37.76714601000003) and POINT (-122.41963999999996 37.777120000000025)
result1 = cg.coordinates(-122.43454601699995, y=37.76714601000003) ## no matched address

Since there are only two geo_points to code and not getting results from censusgeocode.  
Try put the two points into Google Map:  
POINT (-122.43454601699995 37.76714601000003)- 959 14TH ST  
POINT (-122.41963999999996 37.777120000000025)- Van Ness Ave & Hayes St Intersection, next to 150 VAN NESS AVE  
These 11 citations won't really effect the mapping result.

In [17]:
# Assign those addresses to st_cleaning_map dataframe
st_cleaning_map.loc[
    st_cleaning_map['geom'] == 'POINT (-122.41963999999996 37.777120000000025)', 
    'Citation Location'
] = '150 VAN NESS AVE'

st_cleaning_map.loc[
    st_cleaning_map['geom'] == 'POINT (-122.43454601699995 37.76714601000003)', 
    'Citation Location'
] = '959 14TH ST'

In [18]:
# See whole dataframe. Noted: 'Location_new' did not change
st_cleaning_map

,Citation Number,Citation Issued DateTime,Violation,Violation Description,Citation Location,Vehicle Plate State,Vehicle Plate,Fine Amount,Date Added,geom,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods,Issued Year,Location_new
0,949367440,2022-03-23 02:33:00,TRC7.2.22,STR CLEAN,720 TURK ST,CA,8U09234,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"720 TURK ST, SAN FRANCISCO, USA"
1,949375335,2022-03-24 09:30:00,TRC7.2.22,STR CLEAN,19 RIVAS AVE,CA,8GOA162,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"19 RIVAS AVE, SAN FRANCISCO, USA"
2,949395613,2022-03-21 13:05:00,TRC7.2.22,STR CLEAN,1405 BAKER ST,CA,5JZT981,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"1405 BAKER ST, SAN FRANCISCO, USA"
3,949411481,2022-03-29 00:15:00,TRC7.2.22,STR CLEAN,420 NATOMA ST,CA,8DEJ983,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"420 NATOMA ST, SAN FRANCISCO, USA"
4,949412855,2022-03-22 12:25:00,TRC7.2.22,STR CLEAN,717 WALLER ST,CA,6GOF707,84.0,11/16/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"717 WALLER ST, SAN FRANCISCO, USA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373569,957009465,2022-11-08 07:24:00,TRC7.2.22,STR CLEAN,6220 GEARY BLVD,CA,7GWE932,87.0,11/15/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"6220 GEARY BLVD, SAN FRANCISCO, USA"
2373570,957033033,2022-11-09 09:18:00,TRC7.2.22,STR CLEAN,2100 LINCOLN WAY,CA,8MXY663,87.0,11/15/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"2100 LINCOLN WAY, SAN FRANCISCO, USA"
2373571,957011543,2022-11-08 09:05:00,TRC7.2.22,STR CLEAN,315 FUNSTON AVE,CA,8UXY385,87.0,11/19/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"315 FUNSTON AVE, SAN FRANCISCO, USA"
2373572,957032403,2022-11-08 08:52:00,TRC7.2.22,STR CLEAN,2501 25TH AVE,CA,7TRV758,87.0,11/20/2022 12:00:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,2022,"2501 25TH AVE, SAN FRANCISCO, USA"


### Where are the citation hotspots in recent five years?
From last notebook, we know that top 10 hotspots are all close to each location: 5 locations on the Mission St., 3 on Taylor St., and 2 on Jones St.  
If we put the locations on the map, 5 of the locations are in the Tenderloin district, 4 in SOMA district, and 1 in Financial district.

In [19]:
# Top 10 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(10)

150 VAN NESS AVE    4362
1190 MISSION ST      459
111 TAYLOR ST        339
540 JONES ST         327
1188 MISSION ST      287
814 MISSION ST       282
555 MISSION ST       270
556 JONES ST         262
520 TAYLOR ST        251
1018 MISSION ST      249
Name: Citation Location, dtype: int64

In [20]:
# Top 50 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(50)

150 VAN NESS AVE             4362
1190 MISSION ST               459
111 TAYLOR ST                 339
540 JONES ST                  327
1188 MISSION ST               287
814 MISSION ST                282
555 MISSION ST                270
556 JONES ST                  262
520 TAYLOR ST                 251
1018 MISSION ST               249
501 TAYLOR ST                 242
50 CLEARY CT                  239
1000 GREAT HWY                235
3500 19TH ST                  234
1190 MISSION STREET           226
3048 16TH ST                  219
555 ELLIS ST                  216
450 JONES ST                  214
480 ELLIS ST                  202
520 SOUTH VAN NESS AVE        198
355 BERRY ST                  197
420 EDDY ST                   193
230 GOLDEN GATE AVE           189
1825 MISSION ST               188
1036 MISSION ST               188
350 PARNASSUS AVE             186
845 HYDE ST                   185
420 JONES ST                  184
225 TAYLOR ST                 184
575 MISSION ST

In [21]:
# Top 100 hotspots
st_cleaning_map['Citation Location'].value_counts().nlargest(100)

150 VAN NESS AVE    4362
1190 MISSION ST      459
111 TAYLOR ST        339
540 JONES ST         327
1188 MISSION ST      287
                    ... 
1180 POST ST         138
30 DORE ST           138
101 POLK ST          138
222 TAYLOR ST        138
755 BROADWAY         137
Name: Citation Location, Length: 100, dtype: int64

### Hotspots each year?
Citation hotspots becomes more concentrated from 2018 to 2022.

#### 2018
Top 10 hotspots are located in: Golden Gate Park(3), SoMA(1), Financial District(1), Western Addition(4), Lakeshore(1).  
The top location for getting ticket is `1000 GREAT HWY` in Golden Gate Park Neighborhood that close to the beach.

In [22]:
# 2018 full list
map_count_2018 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2018-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2019-01-01')
]
map_count_2018 = map_count_2018['Location_new'].value_counts()
map_count_2018

1000 GREAT HWY, SAN FRANCISCO, USA       179
950 GREAT HWY, SAN FRANCISCO, USA        142
1050 GREAT HWY, SAN FRANCISCO, USA       138
814 MISSION ST, SAN FRANCISCO, USA       121
555 MISSION ST, SAN FRANCISCO, USA       104
                                        ... 
125 MAGNOLIA ST, SAN FRANCISCO, USA        1
1752 LANE ST, SAN FRANCISCO, USA           1
159 DAY ST, SAN FRANCISCO, USA             1
206 BADEN ST, SAN FRANCISCO, USA           1
1933 O'FARRELL ST, SAN FRANCISCO, USA      1
Name: Location_new, Length: 217786, dtype: int64

In [23]:
# 2018 top 10
map_count_2018.nlargest(10)

1000 GREAT HWY, SAN FRANCISCO, USA    179
950 GREAT HWY, SAN FRANCISCO, USA     142
1050 GREAT HWY, SAN FRANCISCO, USA    138
814 MISSION ST, SAN FRANCISCO, USA    121
555 MISSION ST, SAN FRANCISCO, USA    104
50 CLEARY CT, SAN FRANCISCO, USA       89
65 CLEARY CT, SAN FRANCISCO, USA       82
60 CLEARY CT, SAN FRANCISCO, USA       80
750 FONT BLVD, SAN FRANCISCO, USA      76
40 CLEARY CT, SAN FRANCISCO, USA       73
Name: Location_new, dtype: int64

#### 2019
Top 10 hotspots are located in: Western Addition, SoMA, Chinatown, Financial District, Potrero Hill, and Mission.  
All top 10 hotspots are located at the east side of the city.  
The top location for citation is `50 CLEARY CT` at Western Addition Neighborhood.

In [24]:
# 2019 full list
map_count_2019 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2019-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2020-01-01')
]
map_count_2019 = map_count_2019['Location_new'].value_counts()
map_count_2019

50 CLEARY CT, SAN FRANCISCO, USA            132
555 MISSION ST, SAN FRANCISCO, USA           89
400 FULTON ST, SAN FRANCISCO, USA            80
814 MISSION ST, SAN FRANCISCO, USA           76
1200 SAN BRUNO AVE, SAN FRANCISCO, USA       72
                                           ... 
1124 ELLIS ST, SAN FRANCISCO, USA             1
05 MOSS ST, SAN FRANCISCO, USA                1
1169 BAY ST, SAN FRANCISCO, USA               1
3172 CESAR CHAVEZ ST, SAN FRANCISCO, USA      1
1269 SILVER AVENUE, SAN FRANCISCO, USA        1
Name: Location_new, Length: 247365, dtype: int64

In [25]:
# 2019 top 10
map_count_2019.nlargest(10)

50 CLEARY CT, SAN FRANCISCO, USA           132
555 MISSION ST, SAN FRANCISCO, USA          89
400 FULTON ST, SAN FRANCISCO, USA           80
814 MISSION ST, SAN FRANCISCO, USA          76
1200 SAN BRUNO AVE, SAN FRANCISCO, USA      72
318 HOWARD ST, SAN FRANCISCO, USA           60
55 CLEARY CT, SAN FRANCISCO, USA            58
450 RHODE ISLAND ST, SAN FRANCISCO, USA     58
1009 MISSION ST, SAN FRANCISCO, USA         58
973 MISSION ST, SAN FRANCISCO, USA          56
Name: Location_new, dtype: int64

#### 2020

In [26]:
# 2020 full list
map_count_2020 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2020-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2021-01-01')
]
map_count_2020 = map_count_2020['Location_new'].value_counts()
map_count_2020

590 MISSION ST, SAN FRANCISCO, USA     74
110 08TH ST, SAN FRANCISCO, USA        66
111 TAYLOR ST, SAN FRANCISCO, USA      56
1190 MISSION ST, SAN FRANCISCO, USA    51
1018 MISSION ST, SAN FRANCISCO, USA    51
                                       ..
1139 SANCHEZ ST, SAN FRANCISCO, USA     1
460 LISBON ST, SAN FRANCISCO, USA       1
255 MAYNARD ST, SAN FRANCISCO, USA      1
898 CAROLINA ST, SAN FRANCISCO, USA     1
55 EDGAR AVENUE, SAN FRANCISCO, USA     1
Name: Location_new, Length: 223312, dtype: int64

In [27]:
# 2020 top 10
map_count_2020.nlargest(10)

590 MISSION ST, SAN FRANCISCO, USA         74
110 08TH ST, SAN FRANCISCO, USA            66
111 TAYLOR ST, SAN FRANCISCO, USA          56
1190 MISSION ST, SAN FRANCISCO, USA        51
1018 MISSION ST, SAN FRANCISCO, USA        51
540 JONES ST, SAN FRANCISCO, USA           50
1036 MISSION STREET, SAN FRANCISCO, USA    49
10 12TH ST, SAN FRANCISCO, USA             48
355 MCALLISTER ST, SAN FRANCISCO, USA      46
1055 MASON ST, SAN FRANCISCO, USA          44
Name: Location_new, dtype: int64

#### 2021

In [28]:
# 2021 full list
map_count_2021 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2021-01-01') & 
    (st_cleaning_map['Citation Issued DateTime'] < '2022-01-01')
]
map_count_2021 = map_count_2021['Location_new'].value_counts()
map_count_2021

1190 MISSION ST, SAN FRANCISCO, USA        173
1190 MISSION STREET, SAN FRANCISCO, USA    128
556 JONES ST, SAN FRANCISCO, USA           127
1188 MISSION ST, SAN FRANCISCO, USA        105
540 JONES ST, SAN FRANCISCO, USA           104
                                          ... 
300 HOLYOKE STREET, SAN FRANCISCO, USA       1
2121 GOUGH ST, SAN FRANCISCO, USA            1
5426 ANZA STREET, SAN FRANCISCO, USA         1
227 CABRILLO STREET, SAN FRANCISCO, USA      1
7 SHOTWELL ST, SAN FRANCISCO, USA            1
Name: Location_new, Length: 266447, dtype: int64

In [29]:
# 2021 top 10
map_count_2021.nlargest(10)

1190 MISSION ST, SAN FRANCISCO, USA        173
1190 MISSION STREET, SAN FRANCISCO, USA    128
556 JONES ST, SAN FRANCISCO, USA           127
1188 MISSION ST, SAN FRANCISCO, USA        105
540 JONES ST, SAN FRANCISCO, USA           104
555 ELLIS ST, SAN FRANCISCO, USA           103
101 POLK ST, SAN FRANCISCO, USA             99
111 TAYLOR ST, SAN FRANCISCO, USA           96
887 HOWARD STREET, SAN FRANCISCO, USA       93
1188 MISSION STREET, SAN FRANCISCO, USA     92
Name: Location_new, dtype: int64

#### 2022

In [30]:
# 2022 full list
map_count_2022 = st_cleaning_map[
    (st_cleaning_map['Citation Issued DateTime'] >= '2022-01-01')
]
map_count_2022 = map_count_2022['Location_new'].value_counts()
map_count_2022

1190 MISSION ST, SAN FRANCISCO, USA         125
540 JONES ST, SAN FRANCISCO, USA            101
1018 MISSION ST, SAN FRANCISCO, USA         101
111 TAYLOR ST, SAN FRANCISCO, USA            93
450 JONES ST, SAN FRANCISCO, USA             87
                                           ... 
57 TINGLEY ST, SAN FRANCISCO, USA             1
1733 SAN JOSE AVENUE, SAN FRANCISCO, USA      1
155 AVALON AVE, SAN FRANCISCO, USA            1
05 EMBARCADERO NORTH, SAN FRANCISCO, USA      1
2501 25TH AVE, SAN FRANCISCO, USA             1
Name: Location_new, Length: 218677, dtype: int64

In [31]:
# 2022 top 10
map_count_2022.nlargest(10)

1190 MISSION ST, SAN FRANCISCO, USA           125
540 JONES ST, SAN FRANCISCO, USA              101
1018 MISSION ST, SAN FRANCISCO, USA           101
111 TAYLOR ST, SAN FRANCISCO, USA              93
450 JONES ST, SAN FRANCISCO, USA               87
50 JONES ST, SAN FRANCISCO, USA                82
1188 MISSION ST, SAN FRANCISCO, USA            82
520 SOUTH VAN NESS AVE, SAN FRANCISCO, USA     76
1333 JONES ST, SAN FRANCISCO, USA              72
420 JONES ST, SAN FRANCISCO, USA               71
Name: Location_new, dtype: int64